In [12]:
import pandas as pd

In [17]:
# list of seasons to consider
# year_list = ['20142015', '20152016', '20162017', '20172018', '20182019', '20192020', '20202021', '20212022', '20222023', '20232024']
year_list = ['20202021', '20212022', '20222023', '20232024']

In [18]:
import process_data

# find list of teams active between 2014 and 2024
team_id_list = []
for season in year_list:
    for id in process_data.get_season_teams(season):
        if id not in team_id_list:
            team_id_list.append()
team_id_list

AttributeError: module 'process_data' has no attribute 'get_season_teams'

In [ ]:
# find list of player ids for players who played a game between 2010 and 2020
player_id_list = []
for team in team_id_list:
    for season in year_list:
        res = requests.get("https://statsapi.web.nhl.com/api/v1/teams/{}/?expand=team.roster&season={}".format(team, season))
        try:
            for player in res.json()['teams'][0]['roster']['roster']:
                player_id_list.append(player['person']['id'])
        except:
            pass
player_id_list = set(player_id_list)

In [ ]:
# extract data about all players who played between 2010 and 2020, including personal data and stats from every season in the NHL during that period
player_stats = []
for player in player_id_list:
    res_player = requests.get("https://statsapi.web.nhl.com/api/v1/people/{}".format(player))
    if res_player.json()['people'][0]['primaryPosition']['code'] != 'G':
        data = {'name': res_player.json()['people'][0]['firstName'] + " " + res_player.json()['people'][0]['lastName'],
                'position': res_player.json()['people'][0]['primaryPosition']['code'],
                'country': res_player.json()['people'][0]['nationality'],
                'birthday': res_player.json()['people'][0]['birthDate'],
                'id': player,
                'height': res_player.json()['people'][0]['height'],
                'weight': res_player.json()['people'][0]['weight']}
        res_stats = requests.get("https://statsapi.web.nhl.com/api/v1/people/{}/stats?stats=yearByYear".format(player))
        for year_stats in res_stats.json()['stats'][0]['splits']:
            if year_stats['season'] in year_list and year_stats['league']['name'] == 'National Hockey League':
                data[year_stats['season']] = {'goals': year_stats['stat']['goals'],
                                              'assists': year_stats['stat']['assists'],
                                              'pim': year_stats['stat']['pim'],
                                              'games': year_stats['stat']['games'],
                                              'hits': year_stats['stat']['hits'],
                                              'shots': year_stats['stat']['shots'],
                                              'time': year_stats['stat']['timeOnIce'], 
                                              'plus_minus': year_stats['stat']['plusMinus'],
                                              'team': year_stats['team']['name']}
        player_stats.append(data)

In [ ]:
# total number of players extracted
len(player_stats)

In [ ]:
# restrict to players who played at least 3 seasons and 100 total games
reduced_player_stats = []
for temp in player_stats:
    games = 0
    years = 0
    for year in year_list:
        if year in temp.keys():
            games += temp[year]['games']
            years += 1
    if years >= 3 and games > 100:
        reduced_player_stats.append(temp)

In [ ]:
# number of players with at least 3 seasons and 100 games
len(reduced_player_stats)

In [ ]:
# function to extract the stats in the dict for a specific year and return zeros if the year is not there
def get_year_data(player, year, index):
    if year in player.keys():
        return {key+'_'+index:val for key,val in player[year].items()}
    else:
        return {'goals_'+index: 0,
                'assists_'+index: 0,
                'pim_'+index: 0,
                'games_'+index: 0,
                'hits_'+index: 0,
                'shots_'+index: 0,
                'time_'+index: 0,
                'plus_minus_'+index: 0}

In [ ]:
# create data for prediction, with every possible set of 2 consecutive seasons to predict the 3rd one
split_data = []
for player in reduced_player_stats:
    for i in range(8):
        years = year_list[i:i+3]
        if years[2] in player.keys():
            data = {key:player[key] for key in player.keys() if key not in year_list}
            data = {**data, **get_year_data(player, years[0], '1'), **get_year_data(player, years[1], '2')}
            data['season_1'] = years[0]
            data['season_2'] = years[1]
            data['season_3'] = years[2]
            data['ppg_3'] = (player[years[2]]['goals'] +  player[years[2]]['assists']) /   player[years[2]]['games']
            split_data.append(data)

In [ ]:
# transform the data into a pandas dtaframe for processing
df = pd.DataFrame(split_data)
df